In [10]:
import os
import re
import json
import subprocess

from openai import OpenAI
import sys
sys.path.append('../')
from prompts.design_prompt import design_prompt
from prompts.parsing_prompt import parsing_prompt
from prompts.scripting_prompt import scripting_prompt

In [11]:
with open('../config/config.json') as f:
     config = json.load(f)

In [12]:
OPENSCAD_EXEC = config["OPENSCAD_EXEC_PATH"]
FILE_DIR = "../common/bevel"
DESIGN_FILENAME = "gears_design.md"
PARAMETERS_FILENAME = "gear_parameters.json"
SCRIPT_FILENAME = "gears_script.scad"
if not os.path.exists(FILE_DIR):
    os.makedirs(FILE_DIR)

In [4]:
client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [15]:
design_description = f""" 
Suppose there is a one-stage system with two bevel gears and the desired gear ratio of the reduction system is 2:1.
The height of both gears are 20mm, and the pressure angles are both 20 degrees.
The driving gear's pitch angle and helix angle are 45 and 0 degrees located in the spatial coordinate (0, 0, 0).
Based on the given information, computer module number, teeth number of both gears, pitch angle and the coordniate of the driven gear centers in sptial coordinate.
"""

In [16]:
design_messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_description},
    ]

In [21]:
# stop_tokens = ['']
# collected_tokens = []

# res = client.chat.completions.create(
#     model=config['GPT_MODEL'],
#     messages=design_messages,
#     stream=True,
# )

# for chunk in res:
#     temp_tokens = chunk.choices[0].delta.content
#     if temp_tokens not in stop_tokens:
#         collected_tokens.append(temp_tokens)
#     else:
#         chunk_res = "".join(collected_tokens).strip()
#         print(chunk_res)
#         collected_tokens = []

res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=design_messages,
    temperature=0
)

In [22]:
print(res.choices[0].message.content)

To design a one-stage bevel gear system with a gear ratio of 2:1, we need to determine the module, number of teeth for both gears, pitch angles, and the spatial coordinates of the driven gear center.

### Step 1: Gear Ratio and Teeth Number
Given the desired gear ratio (GR) of 2:1, we can express this as:
$$ GR = \frac{N_2}{N_1} = 2 $$

Where:
- \( N_1 \) = Number of teeth on the driving gear
- \( N_2 \) = Number of teeth on the driven gear

Since the gear ratio is 2:1, we can choose a convenient number of teeth for the driving gear (\( N_1 \)) and calculate \( N_2 \) accordingly. Let's assume \( N_1 \) to be 20 (a common standard number for gear teeth), then \( N_2 \) would be:
$$ N_2 = GR \times N_1 = 2 \times 20 = 40 $$

### Step 2: Module Calculation
The module (m) is a measure of the size of the teeth and is related to the pitch diameter (D) and the number of teeth (N) by the formula:
$$ m = \frac{D}{N} $$

However, we do not have the pitch diameters yet. The module is typically c

In [23]:
with open(os.path.join(FILE_DIR, DESIGN_FILENAME), "w") as f:
    f.write(res.choices[0].message.content)

In [6]:
parsing_description = \
f"gear 1 (driving gear): bevel gear, module 2mm, teeth 20, pitch diameter 40mm, pitch angle 45 degrees, helix angle 0 degrees, height 20mm, coordinate (0, 0, 0) " + \
f"pressure angle 20 degrees \n" + \
f"gear 2 (driven gear): bevel gear, module 2mm, teeth 40, pitch diameter 80mm, pitch angle 45 degrees, helix angle 0 degrees, height 20mm, coordinate (40, 0, 40)" + \
f"pressure angle 20 degrees"

In [7]:
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [8]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [9]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
  "gear 1": {
    "gear_type": "bevel",
    "source": "driving",
    "coordinate": {
      "x": 0,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 20,
    "height": 20,
    "pitch_d": 40,
    "helix_angle": 0,
    "pitch_angle": 45,
    "pressure_angle": 20
  },
  "gear 2": {
    "gear_type": "bevel",
    "source": "driven",
    "coordinate": {
      "x": 40,
      "y": 0,
      "z": 40,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 40,
    "height": 20,
    "pitch_d": 80,
    "helix_angle": 0,
    "pitch_angle": 45,
    "pressure_angle": 20
  }
}


In [10]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "w") as f:
    f.write(gears_parameters)

In [11]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided parameters of gears \n" + parameters

In [12]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [14]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
    # temperature=0
)

In [15]:
def script_postprocessing(script):
    # To remove textual perfix or suffix information in the raw script generated by GPT-4
    pattern = re.compile(r'```(?:openscad|scad)(.*?)```', re.DOTALL)
    matches = re.findall(pattern, script)
    clean_script = ""
    if len(matches) > 0:
        for m in matches:
            clean_script = clean_script + m.strip()
    else:
        clean_script = script
    return clean_script

In [16]:
script = scripting_response.choices[0].message.content.strip()
script = script_postprocessing(script)
print(script)

translate([0,0,0]) rotate([0, 0, 0])
gear_bevel(m=2, z=20, h=20, w=20, w_helix=0, w_bevel=45);

translate([40,0,40]) rotate([0, -90, 0])
gear_bevel(m=2, z=40, h=20, w=20, w_helix=0, w_bevel=45);


In [19]:
with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "w") as f:
    f.write("include <gears.scad>")

with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "a") as f:
    f.write("\n" + script)
    
# The result is not ideal

In [13]:
subprocess.Popen([OPENSCAD_EXEC, os.path.join(FILE_DIR, SCRIPT_FILENAME)])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>